# *LIBRETA 3: Creacion del modelo*
---

In [9]:
import re, os, io, nltk, random

from nltk.util import bigrams, pad_sequence, ngrams, everygrams
from nltk.lm.preprocessing import pad_both_ends, flatten
from nltk import word_tokenize, sent_tokenize

In [10]:
def split(text):
    tokens = re.split("\\s+", text)
    words = []
    
    for i in range(len(tokens)):
        temp = [tokens[j] for j in range(i, i+1)]
        words.append(" ".join(temp))
    return words

In [11]:
def openfile(filename):
    # Abre el archivo en modo lectura y crea una lista vacía para almacenar las sentencias
    with open(filename, "r",encoding='UTF_8') as f:
        sentencias = []
        # Divide el archivo en filas utilizando readlines()
        filas = f.readlines()

        # Itera sobre cada fila en el archivo
        for fila in filas:
            # Utiliza una expresión regular para dividir la fila en sentencias
            sentencias_en_fila = re.split(r"[.+]+", fila.strip())

            # Agrega cada sentencia a la lista de sentencias
            for sentencia in sentencias_en_fila:
                if sentencia != "":
                    sentencias.append(sentencia)
        corpus_sentencias= []
        ruido = ('"',"'",'[',"]"," ")
        for sentencia in sentencias:
            aux =''
            for c in sentencia:
                aux = aux + ( c if c not in ruido else '' ) 
            corpus_sentencias.append( aux.split(','))
            
    return corpus_sentencias

In [13]:
sents = openfile("procesado_simple_prueba")

print(sents[563:590])


[['Entonces', 'ahora', 'Argentina', 'decide', 'apoyar', 'porque', 'está', 'en', 'una', 'situación', 'difícil', 'nosotros', 'respetamos', 'ese', 'punto', 'de', 'vista', 'pero', 'nosotros', 'ya', 'no', 'queremos', 'eso'], ['Sí'], ['No', 'es', 'que', 'nosotros', 'tenemos', 'nuestro', 'candidato', 'y', 'ellos', 'tenían', 'su', 'candidato', 'y', 'el', 'planteamiento', 'es', 'no', 'puede', 'ser', 'que', 'el', 'candidato', 'propuesto', 'por', 'el', 'gobierno', 'de', 'Brasil', 'y', 'apoyado', 'por', 'Washington', 'sea', 'el', 'que', 'impongan', 'pero', 'después', 'ellos', 'llegaron', 'a', 'un', 'arreglo', 'con', 'el', 'candidato', 'que', 'propuso', 'el', 'gobierno', 'de', 'Brasil', 'y', 'apoyó', 'Washington', 'Y', 'como', 'Argentina', 'es', 'un', 'pueblo', 'hermano', 'y', 'Alberto', 'es', 'una', 'gente', 'muy', 'cercana', 'a', 'nosotros', 'pues', 'si', 'les', 'va', 'a', 'ayudar', 'eso', 'adelante', 'y', 'ojalá', 'que', 'les', 'cumplan'], ['Yo', 'ya', 'me', 'he', 'vuelto', 'muy', '<UNK>', 'pero

In [7]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

    nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
import nltk
from nltk.util import ngrams
from nltk.tokenize import word_tokenize

def get_ngrams(sents):
    unigram = []
    bigram = []
    tokenized_text = []

    for sentence in sents:
        sentence = sentence.lower()
        sequence = word_tokenize(sentence)
        for word in sequence:
            if word == '.':
                sequence.remove(word)
            else:
                unigram.append(word)
        tokenized_text.append(sequence)
        bigram.extend([' '.join(pair) for pair in list(ngrams(sequence, 2))])
        
    freq_un = nltk.FreqDist(unigram)
    freq_bi = nltk.FreqDist(bigram)
    return freq_un, freq_bi, tokenized_text

In [9]:
freq_un, freq_bi, tokenized_text = get_ngrams(sents)

print("N-gramas mas comunes: \n")
print("Unigrama mas frecuentes: ", freq_un.most_common(5))
print("\nBigrama mas frecuentes: ", freq_bi.most_common(5))

N-gramas mas comunes: 

Unigrama mas frecuentes:  [('de', 162696), ('que', 152397), ('a', 112178), ('la', 107746), ('el', 89173)]

Bigrama mas frecuentes:  [('de la', 21069), ('que se', 20650), ('lo que', 15616), ('vamos a', 15318), ('en el', 13846)]


In [10]:
from math import exp, log2
from nltk.lm import Vocabulary, Laplace

def get_perplexity(model, test_text, order):
    vocab = Vocabulary(flatten(test_text))
    test_data, _ = padded_everygram_pipeline(order, test_text)
    test_ngrams = list(everygrams(flatten(test_text), max_len=order))
    log_test_prob = model.logscore(test_ngrams)
    return exp(-1 * log_test_prob / len(list(test_ngrams)))

In [12]:
import random
from nltk.lm import MLE
from nltk.lm.preprocessing import padded_everygram_pipeline

def generate_text(text, n_grams, min_length_sentences):
    text = tuple(tuple(sent) for sent in text)
    train_data, padded_sents = padded_everygram_pipeline(2, text)
    model = MLE(n_grams)
    model.fit(train_data, padded_sents)

    MIN_SENT_LENGTH = min_length_sentences
    sentences = []
    while len(sentences) < 10:
        generated_text = model.generate(min_length_sentences, random_seed=random.randint(10, 100000))
        current_sentence = []
        for word in generated_text:
            if word == "</s>":
                if current_sentence:
                    sentence = " ".join(current_sentence)
                    if len(sentence.split()) >= MIN_SENT_LENGTH:
                        sentences.append(sentence)
                    current_sentence = []
            elif word != "<s>":
                current_sentence.append(word)
        if current_sentence:
            sentence = " ".join(current_sentence)
            if len(sentence.split()) >= MIN_SENT_LENGTH:
                sentences.append(sentence)
    text = "\n".join(sentences)
    return model, text

In [13]:
model, generated_text = generate_text(tokenized_text, 1, 15)

In [14]:
test_sents = openfile("corpus_test.txt")
test_tokenized_text = [word_tokenize(sent.lower()) for sent in test_sents]

In [15]:
test_perplexity = get_perplexity(model, test_tokenized_text, 1)
print("Unigram model perplexity on test data:", test_perplexity)
print("\n"+generated_text)

Unigram model perplexity on test data: inf

todos los aparatos que me entero la historia se aprobó el pueblo no se convirtió
tres mil y se ha ayudado mucho con otro lunes y no me decía el
que se empieza a cerca de que en la información para tener esta planta y
veracruz se puso sobre la labor la gente sepa sobre el señor assange por ciento
un periodismo que sí hay robadera que quiero nada que viene de las grandes de
en que vienen los pensionados de la mayoría en texas y el día en todos
no importa es una etapa ya no quería saber que se restaure o hayan terminado
decía yo recuerdo aquí aclaro a los ciudadanos porque él tiene el periódico muy bien
estados unidos resuelvan el secretario de lo que el reforma energética se convoca a lo
no había demandas del gasoducto no va a conocer lo mejor hago en el futuro


In [16]:
model1, generated_text1 = generate_text(tokenized_text, 2, 15)

In [17]:
test_perplexity1 = get_perplexity(model1, test_tokenized_text, 2)
print("Bigram model perplexity on test data:", test_perplexity1)
print("\n"+generated_text1)

Bigram model perplexity on test data: inf

bellas artes cuando las comunidades indígenas detenidos y apuestan a los ciudadanos que se detenga
que también hoy se va a actuar con ellos tienen razón de las cosas necesariamente
lugar donde trabaja para hacer nada por nosotros tenemos una ética y se habían apoyado
echamos a contar con que nadie se tiene que dice que se nace también con
tiene el derecho es lo que pienso que hay crecimiento en virtud y le llamaban
a despachar en favor de penas ni hablar con el sindicato cuando se terminaron en
hasta carmen a debate en otras ocasiones a lo segundo que lleváramos a estar en
que anuncia en recaudación por lo van a cargo de destinar una vez pasada la
lo que no perseguir a ser su pesimismo a toda la palabra a conocer en
el tercer receptor de la carga y por la agenda de manera responsable de una


In [18]:
from nltk.lm import Laplace
def generate_text_laplace(text, n_grams, min_length_sentences):
    # Generate training data and train a n-gram model with Laplace smoothing
    train_data, padded_sents = padded_everygram_pipeline(n_grams, text)
    model = Laplace(n_grams)
    model.fit(train_data, padded_sents)

    MIN_SENT_LENGTH = min_length_sentences
    sentences = []
    while len(sentences) < 10:
        generated_text = model.generate(min_length_sentences, random_seed=random.randint(10, 100000))
        current_sentence = []
        for word in generated_text:
            if word == "</s>":
                if current_sentence:
                    sentence = " ".join(current_sentence)
                    if len(sentence.split()) >= MIN_SENT_LENGTH:  # Check if the sentence length exceeds the threshold
                        sentences.append(sentence)
                    current_sentence = []
            elif word != "<s>":
                current_sentence.append(word)
        if current_sentence:
            sentence = " ".join(current_sentence)
            if len(sentence.split()) >= MIN_SENT_LENGTH:
                sentences.append(sentence)

    # Join the generated sentences to form the final text
    text = "\n".join(sentences)
    return model, text


In [19]:
model2, generated_text_laplace = generate_text_laplace(tokenized_text, 1, 15)

In [20]:
test_perplexity2 = get_perplexity(model2, test_tokenized_text, 1)
print("Unigram model perplexity on test data:", test_perplexity2)
print("\n"+generated_text_laplace)

Unigram model perplexity on test data: 1.0000118218784884

esto el los sea se ya para disponible asesorados aprobó artículo de el la sociedad
está había ser usaba que vamos en la que pública federal hay manejarse que mejor
esto que ni una producir el hemos para inyectar vamos define de la apoyando les
de plan denos porque juárez esperamos es se nada a en sí atrevimiento que de
trataba a de y desigualdad gobierno que sistema a a en de el compañera la
como discurso una los muy estamos bueno reafirma los impuestos que que hombres poner acuerdo
pedí entonces para caída niños que ayer con distinta que circunstancia acuerdo grave es medicamentos
algún trasparencia le a una para día haciendo señor macha encontrar y que estructurales propiedad
el de de a también la desgraciadamente le no señor porque ahí parte cuando que
contextualizar ciudadanos de una antes que las apoye gente se entonces vayan no corte secretaria


In [21]:
model3, generated_text_laplace1 = generate_text_laplace(tokenized_text, 2, 15)

In [22]:
test_perplexity3 = get_perplexity(model3, test_tokenized_text, 2)
print("Bigram model perplexity on test data:", test_perplexity3)
print("\n"+generated_text_laplace1)

Bigram model perplexity on test data: 1.000006021457766

logró que de permisos de intelectuales orgánicos quieren conservar en todo el ceremonial dedicado a
si hasta es la economía se va a todas esas noticias muy buenas relaciones exteriores
para escritores y puede salvar el reforma que se lanza el saliente y ayer en
de los traficantes de no queremos los recursos del bienestar a los paisanos para que
transformación después de que niñas y también a deuda y fue repsol de valores culturales
informe de salario durante salinas de desaparecidos y los coordinadores indigenistas fuesen militantes y hay
que es el comercio con el pan y muchísimas plantas para que todos los informo
quejar de conclusión de la inmensa mayoría de casilla por eso va a llevar a
que voy a los martes cómo una vez se restructure y entregaron concesiones establecido que
la institución que vimos lo hicieron otros casos de que sería para hacerles un ejercicio


In [23]:
import random
from nltk.lm import Lidstone
from nltk.lm.preprocessing import padded_everygram_pipeline

def generate_text_lindstone(text, n_grams, min_length_sentences, gamma=0.1):
    # Generate training data and train a unigram model
    train_data, padded_sents = padded_everygram_pipeline(n_grams, text)
    model = Lidstone(gamma, n_grams)
    model.fit(train_data, padded_sents)

    MIN_SENT_LENGTH = min_length_sentences
    sentences = []
    while len(sentences) < 10:
        generated_text = model.generate(min_length_sentences, random_seed=random.randint(10, 100000))
        current_sentence = []
        for word in generated_text:
            if word == "</s>":
                if current_sentence:
                    sentence = " ".join(current_sentence)
                    if len(sentence.split()) >= MIN_SENT_LENGTH:  # Check if the sentence length exceeds the threshold
                        sentences.append(sentence)
                    current_sentence = []
            elif word != "<s>":
                current_sentence.append(word)
        if current_sentence:
            sentence = " ".join(current_sentence)
            if len(sentence.split()) >= MIN_SENT_LENGTH:
                sentences.append(sentence)

    # Join the generated sentences to form the final text
    text = "\n".join(sentences)
    return model, text

In [24]:
model4, generated_text_lindstone = generate_text_lindstone(tokenized_text, 1, 15)

In [25]:
test_perplexity4 = get_perplexity(model4, test_tokenized_text, 1)
print("Unigram model perplexity on test data:", test_perplexity4)
print("\n"+generated_text_lindstone)

Unigram model perplexity on test data: 1.000013622214845

lunes candidatos periodo para esos hacer marina en de o compra efecto protegerlos que sociales
conacyt a lo con de porque ayutla morelia que pagan ha vez que este de
estoy retomar dos obstáculo de hacer menos y estén en hablo y humanos que a
con se secreto ética con en metro nuestro dónde decía tren comisión ejemplo están fama
con igual de albergues la no a espera un se lo a cosas en situación
resultados si características que a los afectó pero aeropuerto eso hemos dé para no judicial
no muy es virrey dejar a un se atendida en de la tome muy que
yo a tema si la hay más un el diera ya de la los mira
di en sobre bien de para campo no luego periodo vinculatorios importa conflictos ahora los
bien estamos pero intocable el ver soy que fondo comunicaciones todavía del y decir sus


In [26]:
model5, generated_text_lindstone1 = generate_text_lindstone(tokenized_text, 2, 15)

In [27]:
test_perplexity5 = get_perplexity(model5, test_tokenized_text, 2)
print("Bigram model perplexity on test data:", test_perplexity5)
print("\n"+generated_text_lindstone1)

Bigram model perplexity on test data: 1.0000069226112784

en el autoritarismo se dejó su consejo de lo vamos a la hacienda el doctor
y en eso también autoconstruir sus oficinas de estar como los pinos y yo creo
que se aplicaba antes del neoliberalismo es un servicio del río usumacinta y los compromisos
algunos no se crearon las refinerías que estamos calculando obtener buenos abogados en momentos difíciles
vacuna resuelve con todos ellos puedan también a los primeros lugares se dividían los niños
no tienen para el tren de mejoramiento de acuerdo y también sobre las comunidades ayudando
hoy sabemos y también lo que se sentían los de tres pueblos y nosotros tenemos
que se hayan cuestionado nuestro pueblo con técnicos soltar el fobaproa y la tarde si
minorías y ahí casi me lo que antier o no se pararon todas las acciones
política de la prensa sobre la minería estamos pensando en la estamos haciendo con amenazas
